In [ ]:
import psycopg2
import psycopg2.extras
import time
import random

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def analysis(target, variable, ensemble):
    conn = psycopg2.connect(
        dbname='ep2_1',
        user='postgres',
        password='!37JzLg+9M2RAu72',
        host='10.90.0.35',
        port=5433
    )
    cursor = conn.cursor()
    start_time = time.time()

    t_scores = {}
    
    sql = f"SELECT count(*) FROM ep2_2_schema.original"
    if ensemble == '16k':
        sql = f"SELECT count(*) FROM public.mesh_16km"
    cursor.execute(sql)
    n = cursor.fetchone()[0]
    
    target_sql = 'UNION \n'.\
        join([f"SELECT cells FROM {k}.summary_ensemble_{ensemble} WHERE variable_id in {target[k]} \n" \
             for k in target.keys()])
    sql = f"WITH pre_target as ({target_sql}),combined_target AS (SELECT DISTINCT cell FROM (SELECT unnest(cells) AS cell FROM pre_target) as foo),\
    target as (SELECT array_agg(cell) as cells, count(cell) as ni FROM combined_target) SELECT * FROM target"
    cursor.execute(sql)
    
    target_cells = cursor.fetchone()
    variable_sql = 'UNION \n'.\
        join([f"SELECT variable_id, cells, occs as nj, '{k}' as node FROM {k}.summary_ensemble_{ensemble} WHERE variable_id in {variable[k]} \n" \
             for k in variable.keys()])
    sql = f"WITH variable as ({variable_sql})\
    SELECT {n} as n, {target_cells[1]} as ni, nj, #(ARRAY{target_cells[0]}::integer[] & variable.cells) as nij, ARRAY{target_cells[0]}::integer[] & variable.cells as int_cells,\
    variable.cells, node, variable_id, get_score(0.0001, nj, #(ARRAY{target_cells[0]}::integer[] & variable.cells), {target_cells[1]}, {n}), \
    get_epsilon(0.0001, nj, #(ARRAY{target_cells[0]}::integer[] & variable.cells), {target_cells[1]}, {n}) FROM variable;"

    try:
        cursor.execute(sql)
        variable_cells = cursor.fetchall()

        #print(variable_cells[0])
        for v in variable_cells:
            for c in v[5]:
                if not c in t_scores.keys():
                    t_scores[c] = v[9]
                else: 
                    t_scores[c] += v[9]
        print(len(t_scores))

        end_time = time.time()
        print(f"Lasted: {end_time - start_time} s")
        cursor.close()
        conn.close()

        return target_cells, variable_cells, end_time - start_time
    except Exception as e:
        print(str(e))
        with open('error_query', 'w') as f:
            f.write(sql)
        cursor.close()
        conn.close()
        return None, None, None

In [ ]:
time_ind = []
time_16k = []
for sp in range(2, 10000):
    print(sp)
    target = {'ep2_2_schema': tuple(set([random.randint(1, 123623) for i in range(sp)]))}
    variable = {
        'ep2_2_schema': tuple(set([random.randint(1, 123623) for i in range(sp)])),
        'ep2_3_schema': tuple(set([random.randint(1, 123623) for i in range(sp)]))
    }
    ensemble = 'ind'
    _, _, t = analysis(target, variable, ensemble)
    time_ind.append(t)
    ensemble = '16k'
    _,_, t = analysis(target, variable, ensemble)
    if t == None:
        print(target, variable)
    else:
        time_16k.append(t)
    

In [ ]:
plt.figure(figsize=(15, 10))

M = [j for j in range(2, len(time_16k)+2)]

plt.plot(M, time_ind, marker='o', linestyle='-', color='b')
plt.plot(M, time_16k, marker='o', linestyle='-', color='r')

plt.title('Performance de nueva propuesta')
plt.xlabel('Especies en target y variables de cada nodo (#)')
plt.ylabel('Tiempo (s)')

plt.grid(True)
plt.xlim(1, len(time_16k)+1)
plt.ylim(0, 60)

plt.savefig('performance.png', dpi=300, bbox_inches='tight')

plt.show()